In [4]:
! pip install transformers torch onnx onnxruntime

In [8]:
import time

import torch
from transformers import AutoModel, AutoTokenizer

# Load the model and tokenizer
model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sample input
text = "Hello, how are you?"

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")

In [9]:
# Measure computation time
start_time = time.time()

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

end_time = time.time()

# Print computation time
print(f"Computation time: {end_time - start_time:.6f} seconds")


Computation time: 0.091151 seconds


In [10]:
import torch
from transformers import AutoModel, AutoTokenizer
import time

# Load the model and tokenizer
model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

text = "Hello, how are you?"

inputs = tokenizer(text, return_tensors="pt")

In [11]:

start_time = time.time()

with torch.no_grad():
    outputs = model(**inputs)

end_time = time.time()

print(f"Computation time: {end_time - start_time:.6f} seconds")

Computation time: 0.019879 seconds


In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import onnx

model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dummy_input = tokenizer("Hello, world!", return_tensors="pt")
torch.onnx.export(model, (dummy_input['input_ids'], dummy_input['attention_mask']),
                  "bert_model.onnx", input_names=['input_ids', 'attention_mask'], output_names=['output'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
import onnxruntime as ort

session = ort.InferenceSession("bert_model.onnx")
inputs = {'input_ids': dummy_input['input_ids'].numpy(),
          'attention_mask': dummy_input['attention_mask'].numpy()}
result = session.run(None, inputs)
print(result)


[array([[[-0.0781372 ,  0.15866569,  0.03995359, ..., -0.28053772,
          0.02479693,  0.4080832 ],
        [-0.20164081,  0.17805085,  0.4184336 , ..., -0.25220168,
          0.36295998, -0.09791572],
        [-0.71557206,  0.6750811 ,  0.60169905, ..., -1.1031796 ,
          0.079708  ,  0.05665781],
        [ 0.05266681, -0.1482827 ,  1.3608793 , ..., -0.45132715,
          0.12743674,  0.26551542],
        [-0.71216214, -0.48147807, -0.14383112, ...,  0.5602157 ,
         -0.10615411, -0.13010693],
        [ 0.99546915,  0.13275957, -0.06205506, ...,  0.24602498,
         -0.6502377 , -0.32960385]]], dtype=float32), array([[-0.8129945 , -0.24699892, -0.72893703,  0.5582256 ,  0.33571222,
        -0.07583874,  0.7851467 ,  0.15255824, -0.5705089 , -0.99971294,
        -0.3183091 ,  0.7643433 ,  0.9549923 ,  0.58006483,  0.90462524,
        -0.60371476, -0.3112676 , -0.5444868 ,  0.3739629 , -0.4197004 ,
         0.5470807 ,  0.9996203 ,  0.05597189,  0.27101344,  0.3869104 ,
    

In [5]:
model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

dummy_input = tokenizer("Hello, world!", return_tensors="pt")
torch.onnx.export(model, (dummy_input['input_ids'], dummy_input['attention_mask']),
                  "tinybert_model.onnx", input_names=['input_ids', 'attention_mask'], output_names=['output'])


config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

In [6]:
import onnxruntime as ort

session = ort.InferenceSession("tinybert_model.onnx")
inputs = {'input_ids': dummy_input['input_ids'].numpy(),
          'attention_mask': dummy_input['attention_mask'].numpy()}
result = session.run(None, inputs)
print(result)


[array([[[-0.6482098 ,  0.24975356, -0.19035622, ..., -0.11330065,
          0.22478032,  0.5554072 ],
        [-0.58489853, -0.20961466,  0.3519856 , ..., -0.39932722,
          1.0324132 , -0.01356764],
        [-0.49930638, -0.3442854 ,  0.0075318 , ..., -0.23643571,
         -0.39777815,  0.3546021 ],
        [-0.61921066, -0.09514831,  0.27534112, ..., -0.278036  ,
          0.70937824,  0.42487353],
        [-0.5467839 , -0.1717233 , -0.03214882, ..., -0.21654654,
          0.68952584,  0.27480847],
        [-0.0420004 , -0.19291514, -0.06208962, ..., -0.24763924,
         -0.4546886 ,  0.35417736]]], dtype=float32), array([[-0.02320887, -0.02684731, -0.01178191,  0.07252531, -0.02908267,
         0.01117832, -0.06556367,  0.03230019,  0.01394199, -0.05811882,
        -0.03733499,  0.01787299,  0.04351581, -0.0644822 ,  0.04152397,
        -0.01188442,  0.00959238,  0.02593195,  0.01048519,  0.04887405,
         0.0079263 , -0.00900354,  0.05417852,  0.02591185, -0.05913463,
    